In [50]:
from imgaug import augmenters as iaa
import numpy as np
import sys
import os
import glob
from keras.preprocessing import image
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy
from scipy import misc
import scipy.io
from keras.preprocessing import image
from imgaug import augmenters as iaa

In [51]:
testing_gt_path = "/Users/karimabedrabbo/Desktop/dris_cropped/dris_cup/testing"
training_gt_path = "/Users/karimabedrabbo/Desktop/dris_cropped/dris_cup/training"
testing_orig_path = "/Users/karimabedrabbo/Desktop/dris_cropped/dris_cropped/testing"
training_orig_path = "/Users/karimabedrabbo/Desktop/dris_cropped/dris_cropped/training"

testing_gt = glob.glob1(testing_gt_path,"*.png")
training_gt = glob.glob1(training_gt_path ,"*.png")

testing_orig = glob.glob1(testing_orig_path,"*.png")
training_orig = glob.glob1(training_orig_path,"*.png")

In [52]:
testing_gt_imgs = [np.array(image.load_img(testing_gt_path + "/" + fname, target_size=(350, 350, 3))) for fname in testing_gt]
training_gt_imgs = [np.array(image.load_img(training_gt_path + "/" + fname, target_size=(350, 350, 3))) for fname in training_gt]

testing_orig_imgs = [np.array(image.load_img(testing_orig_path + "/"+fname, target_size=(350, 350, 3))) for fname in testing_orig]
training_orig_imgs = [np.array(image.load_img(training_orig_path + "/"+fname, target_size=(350, 350, 3))) for fname in training_orig]


In [53]:
def multiple_image_numpy(imgs, rows, cols):
    total = len(imgs)
    new = np.ndarray((total, rows, cols, 3), dtype=np.uint8)

    for i in range(total):
        new[i] = imgs[i]
    
    return new


In [72]:
def run_augmentation(images, heatmaps):
    # Standard scenario: You have N RGB-images and additionally 21 heatmaps per image.
    # You want to augment each image and its heatmaps identically.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    # All augmenters with per_channel=0.5 will sample one value _per image_
    # in 50% of all cases. In all other cases they will sample new values
    # _per channel_.
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            sometimes(iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
                rotate=(-179, 179), # rotate by -45 to +45 degrees
                #order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                #cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                #mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
            sometimes(iaa.GaussianBlur((0, 3.0)) # blur images with a sigma between 0 and 3.0
            ),
            sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)) # blur images with a sigma between 0 and 3.0
            ),
        ],
        random_order=True
    )



    # Convert the stochastic sequence of augmenters to a deterministic one.
    # The deterministic sequence will always apply the exactly same effects to the images.
    seq_det = seq.to_deterministic() # call this for each batch again, NOT only once at the start
    images_aug = seq_det.augment_images(images)
    heatmaps_aug = seq_det.augment_images(heatmaps)
    
    return images_aug, heatmaps_aug

In [73]:
testing_gt_imgs = multiple_image_numpy(testing_gt_imgs, 350, 350)
training_gt_imgs = multiple_image_numpy(training_gt_imgs, 350, 350)
testing_orig_imgs = multiple_image_numpy(testing_orig_imgs, 350, 350)
training_orig_imgs = multiple_image_numpy(training_orig_imgs, 350, 350)

In [74]:
testing_orig_imgs_aug, testing_gt_imgs_aug = run_augmentation(testing_orig_imgs,testing_gt_imgs)
training_orig_imgs_aug, training_gt_imgs_aug = run_augmentation(training_orig_imgs,training_gt_imgs)

In [75]:
def save_images(orig_path, mask_path, orig_imgs, mask_imgs):
    for i in range(len(orig_imgs)):
        orig = misc.toimage(orig_imgs[i])
        orig.save(orig_path + str(i) + ".png")
    for i in range(len(mask_imgs)):
        mask = misc.toimage(mask_imgs[i])
        mask.save(mask_path + str(i) + ".png")

In [76]:
save_images("/Users/karimabedrabbo/Desktop/Drishti_Augmented/dris_cropped/testing/","/Users/karimabedrabbo/Desktop/Drishti_Augmented/dris_cup/testing/",testing_orig_imgs_aug, testing_gt_imgs_aug)
save_images("/Users/karimabedrabbo/Desktop/Drishti_Augmented/dris_cropped/training/","/Users/karimabedrabbo/Desktop/Drishti_Augmented/dris_cup/training/",training_orig_imgs_aug, training_gt_imgs_aug)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  
